In [1]:
import os, sys
# import numpy as np
from functools import partial
import autograd.numpy as np
from autograd import jacobian, grad, primitive
from scipy.stats import multivariate_normal

## <center>                                           Add module paths

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MomentMatching.StateModels import GaussianState
from MomentMatching.baseMomentMatch import UnscentedTransform, TaylorTransform, MonteCarloTransform
from MomentMatching.auto_grad import logpdf
# from MomentMatching.KalmanFilter import KalmanFilterSmoother

In [3]:
x = np.array([2.0, 0.1])

### <center> Define State Distribution

In [4]:
xx_mean = np.array([1.0, 0.0], dtype=float)
xx_sigma = np.array([[1, 0], [0, 1]], dtype=float)
state_distribution = GaussianState(xx_mean, xx_sigma)


### <center>  Define a linear function 

In [5]:
A = np.array([[1, 0],
              [0, 0.1]], dtype=float)
B = np.array([0.0, 0.0], dtype=float)
Z = np.array([[2, 0],
              [0, 0.3]], dtype=float)

In [6]:
def measurement_function(x, A=A, B=B):
    return np.dot(A, x) + B#[:, np.newaxis]

In [7]:
def transition_function(x, A=Z, B=B):
    return measurement_function(x, A=A, B=B)

### <center> Define Taylor transform TT

In [8]:
TT = TaylorTransform(dimension_of_state=2)

In [9]:
TT.numerical_jacobian(f=transition_function, x=xx_mean)

array([[ 2. ,  0. ],
       [ 0. ,  0.3]])

### <center> 

``` python 
def measurement_update(self, node, measurement):

    assert isinstance(node, TimeSeriesNodeForEP)

    measurement_cavity_distribution = node.marginal / node.measurement_factor  # q(x_t) / q_up(x_t)

    new_node = node

    new_node.marginal = self.moment_matching(self.measurement, measurement_cavity_distribution, self.R, measurement)

    new_node.measurement_factor = new_node.marginal / measurement_cavity_distribution

    return new_node


```

In [10]:
meas_jac = jacobian(transition_function)
meas_jac(xx_mean)

array([[ 2. ,  0. ],
       [ 0. ,  0.3]])

In [11]:
factor_mean = np.array([0.0, 0.0], dtype=float)
factor_sigma = 99999 * np.array([[1, 0], [0, 1]], dtype=float)
measurement_factor = GaussianState(factor_mean, factor_sigma)
measurement = np.array([2.0, 0.0], dtype=float)

In [12]:
# logpdf = primitive(multivariate_normal.logpdf)

In [13]:
def data_likelihood(f, mean, cov, measurement ):
    
    meanz = f(mean)   # z = f(x)
    linear_C = jacobian(f)(mean)  # linear factor A 
    covz = linear_C @ cov @ linear_C.T #  predictive covariance sz = var(f(x)) = A * Sigma * A^T
    logZi = logpdf(measurement,meanz, covz)
    return logZi

In [14]:
def data_likelihood2(f, mean1, cov1, mean2, cov2):
    
    meanx = f(mean1)   # z = f(x)
#     print(f'Mean is {meanx}')
    linear_C = jacobian(f)(mean1)  # linear factor A 
#     print(f'The value of jacobian in datalikelihood2 is {linear_C}')
    covx = linear_C @ cov1 @ linear_C.T #  predictive covariance sz = var(f(x)) = A * Sigma * A^T
#     print(f'The value of predicted cov in datalikelihood2 is {covx}')
    logZi = logpdf(mean2, meanx, covx + cov2)
    return logZi

In [15]:
# logZi_func = logpdf
# dlogZidMz_func = jacobian(logpdf, argnum=1)
# dlogZidSz_func = jacobian(logpdf, argnum=2)

# logZi = logZi_func(x=measurement, mean=mz, cov=sz)
# dlogZidMz = dlogZidMz_func(measurement, mz, sz)
# dlogZidSz = dlogZidSz_func(measurement, mz, sz) 

In [16]:
def moment_matching_gaussian(mxni, Sxni, dlogZidMz, dlogZidSz):
    mx = mxni + np.dot(Sxni, dlogZidMz.T)
    return mx

In [17]:
def ep_measurement_update( measurement_function, state_distribution,  z_measurement):
    measurement_cavity = state_distribution / measurement_factor
    logZi = data_likelihood(measurement_function, measurement_cavity.mean, measurement_cavity.cov, measurement)
    dlogZidMz = jacobian(data_likelihood, argnum=1)(measurement_function, measurement_cavity.mean, measurement_cavity.cov, z_measurement)
    dlogZidSz = jacobian(data_likelihood, argnum=2)(measurement_function, measurement_cavity.mean, measurement_cavity.cov, z_measurement)
    mx = measurement_cavity.mean + np.dot(measurement_cavity.cov, dlogZidMz.T)
#     print(dlogZidMz)
#     print(2 * dlogZidSz)
#     vx = measurement_cavity.cov + measurement_cavity.cov @ (2*dlogZidSz - np.dot(dlogZidMz.T, dlogZidMz) ) @ measurement_cavity.cov
    vx = measurement_cavity.cov - measurement_cavity.cov @  (  np.outer(dlogZidMz, dlogZidMz) - 2*dlogZidSz ) @ measurement_cavity.cov.T  
    filtered_distribution = GaussianState(mx, vx)
    return filtered_distribution
    

In [18]:
def ep_back_update( transition_function, state_distribution,  next_state_distribution, factor ):
    back_cavity = state_distribution / factor
    fwd_cavity = next_state_distribution / factor
    arg_tuple = (transition_function, back_cavity.mean, back_cavity.cov, fwd_cavity.mean, fwd_cavity.cov)
    logZi = data_likelihood2 (*arg_tuple)
    dlogZidMz = jacobian(data_likelihood2, argnum=1)(*arg_tuple)
    dlogZidSz = jacobian(data_likelihood2, argnum=2)(*arg_tuple)
#     back_cavity = state_distribution
    mx = back_cavity.mean + np.dot(back_cavity.cov, dlogZidMz.T)
    print(dlogZidMz)
    print(back_cavity.cov @ (-2 * dlogZidSz + np.outer(dlogZidMz, dlogZidMz))  @ back_cavity.cov )
    print(back_cavity.cov)
#     vx = measurement_cavity.cov + measurement_cavity.cov @ (2*dlogZidSz - np.dot(dlogZidMz.T, dlogZidMz) ) @ measurement_cavity.cov
    vx = back_cavity.cov + back_cavity.cov @  ( - np.outer(dlogZidMz, dlogZidMz) + 2*dlogZidSz ) @ back_cavity.cov.T  
    filtered_distribution = GaussianState(mx, vx)
    return filtered_distribution

In [19]:
def measurement_update(self, measurement_function, state_distribution, z_measurement):

    z_mean, z_cov, xz_cross_cov = self.predict(measurement_function, state_distribution)
    # Add measurement Noise R_t
    z_cov = z_cov #+ self.system_model.R.cov

    K = np.matmul(xz_cross_cov, np.linalg.inv(z_cov))

    corrected_mean = state_distribution.mean + np.dot(K, (z_measurement - z_mean))  # equation 15  in Marc's ACC paper
    corrected_cov = state_distribution.cov - np.dot(K, np.transpose(xz_cross_cov))

    filtered_distribution = GaussianState(corrected_mean, corrected_cov)
    return filtered_distribution

In [20]:
def _smoother(self, transition_function, x_distribution, x_next_distribution):
    xx_mean, xx_cov, xx_cross_cov, = self.predict(transition_function, x_distribution)
    # Add transition Noise Q_t
#         xx_cov = xx_cov + self.system_model.Q.cov
    print(f'The predictions are mean: {xx_mean}, cov:{xx_cov}, cross_cov ={xx_cross_cov} ')
    # calculate smoother gain J_t
    J = np.matmul(xx_cross_cov, np.linalg.inv(xx_cov))
    print(f'The gain of smoother is {J}')
    smoothed_mean = x_distribution.mean + np.dot(J, (x_next_distribution.mean - xx_mean))
    smoothed_cov = x_distribution.cov + J @ (x_next_distribution.cov - xx_cov) @ J.T
    smoothed_distribution = GaussianState(smoothed_mean, smoothed_cov)
    return smoothed_distribution

In [21]:
corrected_ans = measurement_update(TT, measurement_function, state_distribution, measurement)

In [22]:
EP_ans = ep_measurement_update(measurement_function, state_distribution, measurement)

In [23]:
EP_ans == corrected_ans

True

In [24]:
EP_back = ep_back_update(transition_function, state_distribution, state_distribution, measurement_factor)

[-0.4  0. ]
[[ 0.800008    0.        ]
 [ 0.          0.08256963]]
[[ 1.00001  0.     ]
 [ 0.       1.00001]]


In [33]:
EP_back.mean

array([ 0.600006,  0.      ])

In [26]:
smoother_ans = _smoother(TT, transition_function, state_distribution, state_distribution)

The predictions are mean: [ 2.  0.], cov:[[ 4.    0.  ]
 [ 0.    0.09]], cross_cov =[[ 2.   0. ]
 [ 0.   0.3]] 
The gain of smoother is [[ 0.5         0.        ]
 [ 0.          3.33333333]]


In [27]:
smoother_ans.cov

array([[  0.25      ,   0.        ],
       [  0.        ,  11.11111111]])

In [28]:
smoother_ans.mean

array([ 0.5,  0. ])

In [29]:
x_cov = np.array([[ 0.0144,  0.0    ], 
                  [ 0.0,     0.09  ]])

In [30]:
J = state_distribution.cov @ Z @ np.linalg.pinv(x_cov)
(state_distribution.cov - x_cov) 
# state_distribution.mean + np.dot(J, (state_distribution.mean - xx_mean))

array([[ 0.9856,  0.    ],
       [ 0.    ,  0.91  ]])

In [31]:
xx_mean

array([ 1.,  0.])

In [32]:
xz_cross_cov @ np.linalg.pinv(z_cov) @ xz_cross_cov.T

NameError: name 'xz_cross_cov' is not defined

In [ ]:
TT.numerical_jacobian(dummy, sz)

In [ ]:
jac = jacobian(dummy)

In [ ]:
jac(sz)